# Megatron XinLi QA 预测 (BPE v2)

## 环境准备

准备运行这个笔记本的 Jupyter kernel(**如果已经准备就绪，不要重复执行！**)：


1. 配置一个 Conda 环境作为 Jupyter Kernel

In [ ]:
%conda env update -f environments/environment-ipy.yml

安装完毕后，为该 Notebook 选择这个 Kernel (名为`Megatron_LM-ipy`)

2. 在Kernel所在 Conda 环境中安装 Apex

需要通过 pip 从 github 下载源代码安装：

In [ ]:
%pip install -v -r requirements/apex.txt

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## 下载 Checkpoints

文件比较大，根据实际情况选择下载，**不要重复下载**

In [2]:
!aws s3 cp --recursive s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/iter_0190000 ./checkpoints/345m-hmwebmix-bpe-v2/iter_0190000

download: s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/iter_0190000/mp_rank_00/model_optim_rng.pt to checkpoints/345m-hmwebmix-bpe-v2/iter_0190000/mp_rank_00/model_optim_rng.pt


In [3]:
!aws s3 cp s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/latest_checkpointed_iteration.txt ./checkpoints/345m-hmwebmix-bpe-v2/latest_checkpointed_iteration.txt

download: s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/latest_checkpointed_iteration.txt to checkpoints/345m-hmwebmix-bpe-v2/latest_checkpointed_iteration.txt


## Environment Variables

- 用哪个/些 GPU?

In [ ]:
%env CUDA_VISIBLE_DEVICES 0

## Importings

In [2]:
import copy
import csv
import json
import math
import os
import random
import sys
import time
from contextlib import closing
from itertools import chain, compress
from functools import partial
from multiprocessing import Pool
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [3]:
args = SimpleNamespace(
    # Model arguments
    num_layers=24,
    hidden_size=1024,
    num_attention_heads=16,
    max_position_embeddings=1024,
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=1024,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type='GPT2BPETokenizer_CN',
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/345m-hmwebmix-bpe-v2/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=128,
)

In [4]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

### 初始化函数/全局变量

In [5]:
tokenizer = None
model = None

def initialize():
    global model, tokenizer

    # Disable CuDNN.
    torch.backends.cudnn.enabled = False

    # Pytorch distributed.
    initialize_distributed(args)

    # Random seeds for reproducability.
    set_random_seed(args.seed)

    # get the tokenizer
    tokenizer = prepare_tokenizer(args)

    # Model, optimizer, and learning rate.
    model = setup_model(args)

    args.device = torch.cuda.current_device()

    # setting default batch size to 1
    args.batch_size = 1

    assert mpu.get_model_parallel_rank() == 0

### 主进程初始化

In [6]:
%%time

initialize()

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 336128000
global rank 0 is loading checkpoint ./checkpoints/345m-hmwebmix-bpe-v2/iter_0190000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/345m-hmwebmix-bpe-v2/iter_0190000/mp_rank_00/model_optim_rng.pt
CPU times: user 11.4 s, sys: 3.83 s, total: 15.2 s
Wall time: 15.2 s


## Inference functions

In [7]:
def infer_tokens_generative(context_tokens, model, tokenizer):
    context_length = len(context_tokens)
    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)   
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        yield ids[-1]


def infer_text_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s


## 预测试试看

In [11]:
contex_text = '我喜欢'

print(contex_text)
print('==========')
print()

N = 3
for i in range(N):
    print(f'({1+i})/{N}\t', end='')
    for s in infer_text_generative(contex_text.strip(), model, tokenizer):
        print(s, end='')
    print()
    print()

我喜欢

(1)/3	水稻田里的那句话:“种瓜得豆,种豆得瓜,日长一粒长一粒。”不愿意承认长粒需要天,更不愿“踏破铁鞋无觅处,得来全不费功夫。”永远明白自己能干什么,可以干什么的人,永远不会失去个性,永远不会消沉,会一直进步着。是一个没有与自己厮守的人,不会感到寂寞,在寂寞中还能设身处地的走入别人的漩涡深处的存在,把注意力更多的专注于帮助别人。与此相反,另一种人始终喜欢创造无限的可能,即使是最不起眼的,甚至简单的黑暗,也要让自己闪烁的出奇。“舍得”是我们中国人俗话的表述。我们很乐见这样的场景或实际发生的事来:一个人孤身一个人来这里种田,午间的阳光照在他身上,白而没有光芒,仿佛是遍地凋零的叶子,顺着风走,狼狈逃窜,灵魂跟着她越是投奔黑暗的地方,越是显得恍惚。这时候,他旁边墙角上的一棵杂草开始盛开着,边上有一只狗,悄悄的爬了过来,嗅着人的气息,怎么刚刚,这里只有这个人一个人在种田

(2)/3	游览这火车之旅,火车之旅有点像我抱着一沓厚厚的履历书,从东部到西部,一点一滴发生的故事。Swohr敢于提闪亮的名字,也许会让湮没在光芒中的我信心满满。梭罗湖不仅是野生动植物的神奇之地,也是摄影大师王家卫的作品高峰地。这是渡过了一切,世与喧嚣、俗与庸碌,尘世的喧嚣在夜幕中复苏,桥底的小渔船独上风月楼。这一切都来不及在意,可是自己却难以抗拒。在火车上,你能十分真切地看到火车,200一米宽的屋顶莫非能拍出2000cm以上的高速高崎透视吗?Sohr不只是闪耀着法国蓬勃的工业发展,成群的猪狗和Tuesday标签模特,才是它最大的标签。每次一节车厢的人,男人大多清瘦,女人细很多像中国新疆少数民族少女,或五十几岁的形象、干脆鲁莽,来去匆匆,觉察不到怜爱和爱惜。也许她们都很美、但仍清楚地做着坐着的优雅,或说着就

(3)/3	温暖的感觉,喜欢细雨滋润的空间,喜欢聚集在一起打闹的感觉。特别是雨天,听拿捏的焦透的雨声,心情舒畅鸟自在天。下雨天,仿佛是我的任何朋友。要说怎样才会有一个最可爱但是又很自私的朋友,出外面玩和吃饭唯一的一个快乐。在屋顶上,看到雨了。而我想意即我之下,这就好像到了户外呢。越下雨,狂风越大,一派大雨如注漫天的来了。小鸟在飞,不论不用伞的,都一下子飞了出去。哇趴,我会唱歌给你听,这个地方只有我们两个人。嘿嘿,是不是很富日记了。下雨天,我第一个想到的是雨。下雨天,雨水会更肆意的打在身

## 对一组心理/情绪相关文字进行续写

In [12]:
input_texts = [
    '宝贝“啃指甲”是为什么呢？宝贝是缺铁、缺锌了吗？宝贝是肚子里有虫吗？这指甲里好多细菌宝宝都吃下去怎么是好？怎么能让宝宝不“啃指甲”呢？家长应该怎么应对呢？',
    '几年前，英国就有专家向国会递交报告，建议政府制定法律禁止家长让年龄低于3岁的儿童看电视。奇幻小说《查理和巧克力工厂》有句台词：“千万、千万、千万别让孩子，靠近你的电视，最好是别购买、安装，这最最愚蠢的东西”。',
    '婚姻是一座炼狱，因为在这场婚姻里夹杂着各种人际关系，经过这些人际关系，人才能慢慢心智成熟起来。而能够把这些人际关系处理好的，往往都是生活中的高手。那么夫妻双方该如何经营婚姻呢？这里就提供一些方法供参考。',
]

In [13]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 2

for txt in input_texts:
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(txt)
    print()
    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(txt, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

宝贝“啃指甲”是为什么呢？宝贝是缺铁、缺锌了吗？宝贝是肚子里有虫吗？这指甲里好多细菌宝宝都吃下去怎么是好？怎么能让宝宝不“啃指甲”呢？家长应该怎么应对呢？

0: 我们知道婴幼儿时期的宝宝比较喜欢吮吸大人手指的奶嘴或者手指的玩具,特别是玩具的碎末,如果宝宝经常的大口或过快的吮吸和吮吸手指而且常常还会吮到头发上,就表明该幼儿可能存在肠胃疾病或者有碍健康的可能性。如果非常频繁的长指甲会导致指甲短小变厚或者揉捏不自主,甚至会啃掉手指甚至指甲根部位,影响手指的正常形状和形状的保持和指甲的生长。重要的是当宝宝吃手指的时候就不能一直吮吸指甲,应喂小些。而且,孩子的指甲长得粗大和弯曲的时候可能是在尝试某种新的玩具。总之,宝宝的指甲以小指尖长、圆形为好,如果指甲呈尖尖的形状就不行。在我小的时候,我也啃指甲过的,因为喜欢,特别让人痛心,我还屡试不爽。孩子还会啃指甲,就是如果孩子指头粘在大人的手背上或者盘子里,不妨就用指甲刀把孩子的指甲割下来,平时也不要让孩子接触任何器皿。如果是自己的吃的,建议给宝宝多喝水吃水果,多晒太阳,多和爸爸妈妈在家运动为好。带孩子去查

1: 历史原因艾滋病病毒可以在粘土中生存,当温度降低时生命便会结束。沿海地区的婴儿,忽然生病或医学上的疾病,比其他地区婴儿得病更快,死亡率低。目前在你的各儿子手腕上各用手腕毛细血管。3.伤口艾滋病:吞噬伤口中的炮球菌,对病情十分严重,可造成真正的重症。许多婴儿死亡的内因是:多喝水。许多婴儿在婴儿期咳嗽或发烧时,经常咳出很黏稠的皮疹。后来发现这些东西里有“琴弦”在转移。治疗方法:立即将孩子病人的伤口涂擦“琴弦”溶液。洗内裤时,避免使用泻药。患儿眼睛很易于黏上清洁的手纸,一滴清水点入便可消失。目前在你的各儿子手腕上各用手腕毛细血管。药理治疗肌注可显著降低患儿脑电图反应。很小的婴儿在脐带缠成的管内会发生塌陷性呼吸,几分钟至一周左右就会完全停止。约有82%的病例之抑制该现象有效。预防措施:遵守医院的操作规程,



几年前，英国就有专家向国会递交报告，建议政府制定法律禁止家长让年龄低于3岁的儿童看电视。奇幻小说《查理和巧克力工厂》有句台词：“千万、千万、千万别让孩子，靠近你的电视，最好是别购买、安装，这最最愚蠢的东西”。

0: 但遗憾的是,家长们直到今天才幡然醒悟。孩子视力下降,跟电视在生活中的作用没多大关系。向孩子喜欢的电视正式购

## Test

使用 test 语料，从中随机打断，并预测下文，比较原文与预测结果！

随机选 N 个

In [8]:
N = 1000

input_file = './data/hmwebmix/hmwebmix.test.json'
output_file = f'./data/hmwebmix/hmwebmix.test.{args.out_seq_length}.{N}.tsv'

total = sum(1 for _ in tqdm(open(input_file)))
print(f'Test 数据总数: {total:,d}')

assert total >= N

print(f'Test 采样数: {N:,d}')

mask = np.zeros(total, dtype=int)
mask[:N] = 1
np.random.shuffle(mask)

with open(input_file) as input_fp, \
     open(output_file, 'w') as output_fp:
    reader = compress(input_fp, mask)
    writer = csv.writer(output_fp, delimiter='\t')
    for line in tqdm(reader, total=N):
        line = line.strip()
        if not line:
            continue
        context_txt = json.loads(line)['text']
        context_ids = tokenizer.EncodeAsIds(context_txt).tokenization
        context_len = len(context_ids)
        idx = round(random.gauss(context_len*0.5, context_len*0.1))
        input_ids = context_ids[:idx]
        label_ids = context_ids[idx:]
        infer_ids = [id_ for id_ in infer_tokens_generative(input_ids, model, tokenizer)]
        row = [
            tokenizer.DecodeIds(ids)
            for ids in (input_ids, label_ids, infer_ids)
        ]
        writer.writerow(row)


10000it [00:00, 107360.44it/s]
  0%|          | 0/1000 [00:00<?, ?it/s]

Test 数据总数: 10,000
Test 采样数: 1,000


100%|██████████| 1000/1000 [1:04:12<00:00,  3.85s/it]


## 预测一组 QA

In [ ]:
input_texts = [
    (
        '最近晚上睡不着觉，但很困，总觉得不会睡觉？',
        '最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心'
    ),
    (
        '大一，谈了将近一年的女友突然提出分手我该怎么办？',
        ' 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。'
    ),
    (
        '男人出轨',
        '我的意思是，出轨者会问自己，我为什么要这么做，被背叛的伴侣会问，你为什么要这样对我?'
    ),
    (
        '我的奇葩婚姻，老公说为了孩子好就是不离婚？',
        '我的婚姻最近好像一塌糊涂，可是老公不离婚，我甚至有想过死？我是不是得了抑郁症？和老公从2017年开始就吵吵闹闹，一吵架就分居，分居一两个月老公可以对我和孩子不问不闻，到时间了他就自己又搬回来，但是回来也不和我沟通，就算是上次是他动手，他也不道歉，他特别喜欢冷战。现在和老公又发展到了无性，而且双方家庭也合不来，我和他家人不敢来往，他家人对我要求高，希望天天在家做饭带娃伺候老公，他现在和我父母也不来往，从来不回家吃饭，要等我父母走了他才回来，每天很晚回家，去哪从来不告诉我，我知道他有婚外性，但他從來不承認，还说我是神经病，我還被他傳播上了高危型的HPV，以后是死是活都不确定，我特别想要离婚，我感觉和他生活快要窒息了，他整天拉着个脸，要么对我不闻不问，要么一开口说话就是责怪的语气，但他就是不离婚，说是为了孩子好，还让我不要发神经，我真的不知道该怎么和他相处了。他性格很古怪'
    ),
    (
        '妻子在钱的问题上不坦诚，如何处理好家庭经济问题？',
        '和老婆因为意外怀孕结婚，两家谈婚论嫁时闹得不愉快，我家出了一套大户型的房子，一台豪华车（当然这些都是婚前财产），然后给了40万装修婚房，同时给女方10万彩礼。而女方一开始的态度是一分钱没有，后来迫于压力给了50万现金，我老婆名下既没有房也没有车。婚后的钱一直都是给我老婆管理，包括我的工资奖金还有结婚的份子钱等等。我老婆并没有把这些钱看做是家庭财产，就算其中已经掺入了很多我的钱，还是认为全部都是她自己的钱，我只是知道她的卡密码而已，但是卡在哪里，她花了多少钱，花在哪里了，有没有借钱给别人，我通通不知道，这让我很不安。况且她没有理财观念，我曾经三番五次跟她说了理财的重要性，她就只是沉默不语，也不知道是听不懂，还是装傻实际上自己去做了理财但是没把收入情况告诉我，我们多次吵架都是为了钱，我觉得夫妻两个在这个问题上不能坦诚是个大问题，我现在很想经济分开，但是不知道怎么做才比较妥当'
    ),
    (
        '30岁男生，相亲屡次不顺，我该怎么办呢？',
        '30岁男生还单身，相亲对象不少，可是次次都以失败收场，自己长相还行，就是不太会说话，和女生聊天刚开始聊的挺不错，没聊几天女的就不回复了，爱答不理，心里郁闷的晚上都想哭，我该怎么办'
    ),
    (
        '30岁女生，离婚念头挥之不去，他为何计划婚内出轨？',
        '此刻有点儿失眠心情烦躁，我想了很久没有想明白，为什么他会计划着婚内出轨，……心里，我基本上认定原因应该是我身材不怎么好，胸部不够吸引人，因为婚后发现了在婚前，他约P了一个又肥又长相不佳的女人。我以为就是因为对方胸部傲人。在壹心理一些文章的引导下，我突然明白，源头是性生活不和谐。婚前同居一年多，到婚后2年，性生活基本就是，直奔主题，他一手拿着手机欣赏着他的成人影片一边爱爱，很少会主动关心我的感受，也很少理会我让他收起手机的要求。而且另一方面，他基本每晚都会要求我用嘴亲亲，他自己则拿着手机看片或者无关紧要的东西，让我经常觉得自己就是一个娃娃一个工具。以前我主动跟他说起过几次，然而也只是不了了之。后来我基本对性生活没有兴趣，也很反感为他口J。而且，日常生活变得频繁小吵冷战，很少有之前无话不谈的亲密，我不知道我们还有没有重归于好的机会……'
    ),
    (
        '29岁哺乳期，和公婆因带孩子问题每天崩溃，怎么办？',
        '怀孕她就没怎么管，备产各种东西都是自己准备。产后直接是妈妈照顾的。产后半年回来要上班，我爸妈又没退休，无奈下还是他们来照顾。他们方法不对，连个衣服都洗不干净，更不用提孩子的饭了。每天我都自己趁下班时间给孩子准备饭和各种东西。最怕的是习惯，比如她喜欢喂孩子各种炒菜，我说好几次一岁之内吃盐不好就是不听，还故意喂。或者孩子的奶瓶，经常不盖盖子让他把玩，说不卫生，也不搭理。孩子吃饭的碗和勺子永远是扔在各种桌子椅子角落，用的时候直接拿起来用也不洗，洗脸洗脚水每次告诉温度差不多就好了，就是不听，偏要一会冷一会烫。来我家被子也不叠，锅永远是不会洗的，下一顿加水加米接着煮。我老公又每天忙的不在家，我已经要崩溃了'
    ),
    (
        '婚后分居三四年，有过家庭矛盾，我的婚姻是否该继续？',
        '婚后分居三四年，和老公之间发生过一些家庭矛盾，可以看我另外一篇疑问。我们上半年闹离婚，后来他又主动和好，但是我们之前除了新婚，几乎很少有夫妻生活，他说一靠近我就觉得心里像一堆蚂蚁抓挠，特别难受。他说这是他得心理问题，小时候受到唾骂和凌辱，造成了心理阴影。我们俩都觉得生活的很痛苦，但为了孩子没有离婚。我跟他在一起就觉得拘束，不自在。向往的温馨家庭生活也得不到，我很累，我应该坚持离婚吗？他这种心理疾病是真的还是借口呢？'
    ),
#     (
#         '',
#         '新 闻 大S自嘲上辈子应是儿子的“杀父仇人”，笑说：“不然我怎么一生他就快死了，他是来报仇的吧，而且我一抱他就哭，到三个月后才好一点。”她生下一双儿女集满“好”字，感到非常满足，是否再拼第三胎？大S笑说：“不可能！我现在尽量和老公分房睡。” 评析起初，大家并不看好大S与汪小菲这段婚姻。姐弟恋是一回事儿，更多还是觉得有些不够门当户对。如今，女儿与儿子陆续出生。凑足“好”字的两人，幸福得就像花儿开放。俏江南的变化，对大S与汪小菲毫无影响。男的负责赚钱养家，女的负责照顾孩子开心美满。'
#     )
]


In [ ]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 2

for title, text in input_texts:
    txt = f'{title}<sep>{text}<|endoftext|>'
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(title)
    print(text)
    print()
    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(txt, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

## 手动续写测试

定义问题

In [ ]:
question = (
'''
婚后分居三四年，有过家庭矛盾，我的婚姻是否该继续？
''',
'''
婚后分居三四年，和老公之间发生过一些家庭矛盾，可以看我另外一篇疑问。
我们上半年闹离婚，后来他又主动和好，但是我们之前除了新婚，几乎很少有夫妻生活，他说一靠近我就觉得心里像一堆蚂蚁抓挠，特别难受。他说这是他得心理问题，小时候受到唾骂和凌辱，造成了心理阴影。
我们俩都觉得生活的很痛苦，但为了孩子没有离婚。我跟他在一起就觉得拘束，不自在。
向往的温馨家庭生活也得不到，我很累，我应该坚持离婚吗？
他这种心理疾病是真的还是借口呢？
'''
)

**这在里写我的回答**

In [ ]:
# 在这里写一部分
answer = '''
我觉得这是你
'''


**执行下面的 cell ，帮我写把上面这句写完，或者写下一句**

In [ ]:
n_pred = 3

for i in range(n_pred):
    
    txt = '<sep>'.join(s.strip() for s in question) + '<|endoftext|>' + '<bos>' + answer.strip()
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    
    print(f'{i+1}: ', end='')
    s_pred = ''
    with closing(
        infer_text_generative(txt, model, tokenizer)
    ) as generator:
        for s in generator:
            if s:
                print(s, end='')
                if s[-1] in ('.。!！?？'):
                    break
    print()